In [18]:
#import modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale

%matplotlib inline
plt.style.use('ggplot')

In [2]:
df = pd.read_csv('data/whisky.csv')

In [3]:
df.head()

,RowID,Distillery,Body,Sweetness,Smoky,Medicinal,Tobacco,Honey,Spicy,Winey,Nutty,Malty,Fruity,Floral,Postcode,Latitude,Longitude
0,1,Aberfeldy,2,2,2,0,0,2,1,2,2,2,2,2,\tPH15 2EB,286580,749680
1,2,Aberlour,3,3,1,0,0,4,3,2,2,3,3,2,\tAB38 9PJ,326340,842570
2,3,AnCnoc,1,3,2,0,0,2,0,0,2,2,3,2,\tAB5 5LI,352960,839320
3,4,Ardbeg,4,1,4,4,0,0,2,0,1,2,1,0,\tPA42 7EB,141560,646220
4,5,Ardmore,2,2,2,0,0,1,1,1,2,3,1,1,\tAB54 4NH,355350,829140


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RowID       86 non-null     int64 
 1   Distillery  86 non-null     object
 2   Body        86 non-null     int64 
 3   Sweetness   86 non-null     int64 
 4   Smoky       86 non-null     int64 
 5   Medicinal   86 non-null     int64 
 6   Tobacco     86 non-null     int64 
 7   Honey       86 non-null     int64 
 8   Spicy       86 non-null     int64 
 9   Winey       86 non-null     int64 
 10  Nutty       86 non-null     int64 
 11  Malty       86 non-null     int64 
 12  Fruity      86 non-null     int64 
 13  Floral      86 non-null     int64 
 14  Postcode    86 non-null     object
 15  Latitude    86 non-null     int64 
 16  Longitude   86 non-null     int64 
dtypes: int64(15), object(2)
memory usage: 11.5+ KB


In [5]:
# feature engineering
df.drop(['RowID', 'Postcode', 'Latitude', 'Longitude'], axis=1, inplace=True)

In [6]:
df.isnull().sum()

Distillery    0
Body          0
Sweetness     0
Smoky         0
Medicinal     0
Tobacco       0
Honey         0
Spicy         0
Winey         0
Nutty         0
Malty         0
Fruity        0
Floral        0
dtype: int64

In [7]:
labelEncoder = LabelEncoder()
labelEncoder.fit(df['Distillery'])
df['Distillery'] = labelEncoder.transform(df['Distillery'])

In [8]:
data = scale(df)
data

array([[-1.71202643, -0.07542547, -0.40765084, ...,  0.31606376,
         0.25509887,  0.35566418],
       [-1.67174345,  1.00567291,  0.99466805, ...,  1.91497456,
         1.54559904,  0.35566418],
       [-1.63146048, -1.15652385,  0.99466805, ...,  0.31606376,
         1.54559904,  0.35566418],
       ...,
       [ 1.63146048, -2.23762223,  0.99466805, ...,  0.31606376,
        -1.0354013 ,  0.35566418],
       [ 1.67174345, -0.07542547, -0.40765084, ..., -1.28284703,
        -2.32590146, -1.99719114],
       [ 1.71202643, -0.07542547,  0.99466805, ...,  0.31606376,
         0.25509887, -0.82076348]])

In [16]:
target = df[['Distillery']]
features = df.drop('Distillery', axis=1)

In [20]:
# Build cluster model
kmeans = KMeans(n_clusters=3, random_state=1)
kmeans.fit(data)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=1, tol=0.0001, verbose=0)

In [23]:
colors = np.array(["Red","Green","Blue"])
plt.subplot(1,2,1)
plt.scatter(x=df["Body"] ,y= df["Distillery"],c = colors[target],s=50)
plt.title("Before K Means Classificaion")

IndexError: index 3 is out of bounds for axis 0 with size 3

# References

https://www.datacamp.com/community/tutorials/k-means-clustering-python

https://www.datasciencelearner.com/k-means-clustering-in-python-label-dataset/